In [1]:
import requests
import pandas as pd
from collections import defaultdict
import time

/Users/jose/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
LEAGUE_IDS = {
    2023: "994356486438477824",
    2024: "1064047033306136576",
    2025: "1180226065723957248"
}


In [3]:
def fetch_with_retry(url, pause=1.0, max_retries=3):
    for attempt in range(max_retries):
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        time.sleep(pause)
    response.raise_for_status()


In [4]:
players_data = fetch_with_retry("https://api.sleeper.app/v1/players/nfl")
players_df = pd.DataFrame(players_data).T


In [5]:
players_df = players_df.reset_index().rename(columns={"index": "player_id"})
players_df = players_df[["player_id", "position"]]


In [6]:
players_df

,player_id,player_id,position
0,6462,6462,TE
1,11255,11255,OL
2,8842,8842,CB
3,7926,7926,TE
4,1875,1875,LB
...,...,...,...
11397,4810,4810,OT
11398,918,918,TE
11399,1591,1591,G
11400,6744,6744,WR


In [7]:
records = []

for season, league_id in LEAGUE_IDS.items():
    rosters = fetch_with_retry(f"https://api.sleeper.app/v1/league/{league_id}/rosters")
    roster_owner = {r["roster_id"]: r["owner_id"] for r in rosters}

    for week in range(1, 16):
        matchups = fetch_with_retry(f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week}")

        # Players who played this week
        played = set()
        for m in matchups:
            played.update(m.get("starters", []))
            played.update(m.get("players", []))

        for r in rosters:
            owner = roster_owner[r["roster_id"]]
            for pid in r.get("players", []):
                # Flatten in case pid is a list or None
                if isinstance(pid, list):
                    for sub_pid in pid:
                        if not sub_pid:
                            continue
                        if sub_pid not in played:
                            pos_series = players_df.loc[players_df["player_id"] == sub_pid, "position"]
                            pos = pos_series.values[0] if len(pos_series) > 0 else "UNK"
                            records.append({
                                "season": season,
                                "week": week,
                                "owner": owner,
                                "player_id": sub_pid,
                                "position": pos
                            })
                else:
                    if not pid:
                        continue
                    if pid not in played:
                        pos_series = players_df.loc[players_df["player_id"] == pid, "position"]
                        pos = pos_series.values[0] if len(pos_series) > 0 else "UNK"
                        records.append({
                            "season": season,
                            "week": week,
                            "owner": owner,
                            "player_id": pid,
                            "position": pos
                        })


ValueError: Cannot index with multidimensional key